# Implementierung eines einfachen Tokenizers

Ziel dieser Übung ist es, diverse Texte einzulesen, diese zu tokenisieren und als Folge von Tokens auszugeben. Hierbei liegt der Fokus nicht auf Vollständigkeit! Die Eingabetexte müssen nicht *perfekt* tokenisiert werden, nur so gut wie möglich verarbeitet werden.

## 1. Einlesen und Vorverarbeitung der Texte

Im Ordner `tokenizer_data` befinden sich mehrere Dateien mit Eingabetexten in drei verschiedenen Eingabeformaten. Die ersten Schritte sind nun, diese Dateien einzulesen, die Texte zu extrahieren und von unerwünschten Elementen zu säubern, sowie die reinen Texte gesammelt in eine dafür geeignete Datenstruktur zu legen.

`cmc_train_whats_app.txt` und `cmc_train_wiki_discussion_2.txt` sind einfache Textdateien, die durch Leerzeilen getrennte Einzeltexte enthalten. `cmc_train_whats_app.txt` enthält einzelne WhatsApp-Nachrichten. `cmc_train_wiki_discussion_2.txt` enthält einzelne Beiträge von Wikipedia-Diskussionsseiten. Jeder Text hat dabei eine Metadatenkopfzeile in einem XML-ähnlichen Format. Diese Kopfzeile soll entfernt werden, und die einzelnen Texte sollen getrennt bereitgestellt werden.

`epub_alice1.xml` und `epub_mittelpunkt3.xml` enthalten jeweils ein einzelnes Kapitel aus einer E-Book-Ausgabe von Lewis Carrolls "Alice's Adventures in Wonderland" und Jules Vernes "Reise nach dem Mittelpunkt der Erde". Die Kapiteldateien selbst liegen als XHTML-Dokumente vor und stammen so unverändert aus den ursprünglichen Epub-Dateien, aus denen sie extrahiert wurden. Bei diesen Dateien sollen 
1. alle Absätze des eigentlichen Textes extrahiert werden,
2. der Text der Absätze von allen XML-Tags und Zeilenumbrüchen befreit werden,
3. die einzelnen Absätze durch Zeilenumbrüche getrennt zusammengefügt werden.

`twitter-corpus.csv` enthält Tweets als CSV-Tabelle. Hier sollen aus der Spalte "TweetText" die einzelnen Tweets als Texte bereitgestellt werden. Zum Einlesen der CSV-Datei kann entweder wieder Pandas mit der Funktion `read_table()` oder das `csv`-Modul aus der Python-Standardbibliothek verwendet werden (siehe Übung 3.1).

Es bietet sich an, die einzelnen Texte in einer Liste oder einem Dictionary abzuspeichern.

In [1]:
# Imports
import re

"""Liste an Dateinamen der einzulesenden Dateien"""
filenames = [
    'tokenizer_data/cmc_train_whats_app.txt',
    'tokenizer_data/cmc_train_wiki_discussion_2.txt',
    'tokenizer_data/epub_alice1.xml',
    'tokenizer_data/epub_mittelpunkt3.xml',
    'tokenizer_data/twitter-corpus.csv', 
]

In [2]:
"""Diese Funktion liest die Texte aus den cmc_*.txt-Dateien ein und gibt sie als Liste zurück"""
def extract_txt(filename):
    texts = []
    return texts

In [3]:
"""Diese Funktion liest die Texte aus den epub_*.xml-Dateien ein und gibt sie als Liste zurück"""
def extract_epub(filename): 
    texts = []
    return texts

In [4]:
"""Diese Funktion liest die Texte aus der twitter-corpus.csv-Datei ein und gibt sie als Liste zurück"""
def extract_twitter(filename):
    texts = []
    return texts

In [5]:
### Hier Code vervollständigen

# Hier sollen die Dateien eingelesen und die einzelnen Texte in eine geeignete Datenstruktur abgelegt werden

## 2. Tokenisierung der Texte

Die im vorherigen Schritt gewonnenen Texte sollen nun tokenisiert werden. Dazu müssen anhand des untenstehenden Code-Gerüsts die regulären Ausdrücke bzw. Token-Klassen zum Tokenisieren vervollständigt werden.

Eine Token-Klasse bezieht sich auf eine Art von Eingabe, die gesondert behandelt werden muss/kann. Typische Beispiele für gesonderte Token-Klassen sind bspw. Zahlen, URLs und "normale" Wörter. 
Die Token-Klassen und regulären Ausdrücke liegen dabei in einer Hierarchie, da sie der Reihe nach durchlaufen werden. Es ist wichtig, dass dabei die speziellen Token-Klassen in der Hierarchie über den generischen stehen, sprich als erstes angegeben werden. Dadurch, dass in jedem Tokenisierungsschritt die gefundenen Tokens aus dem Eingabetext "konsumiert" werden, könnten sonst generischere Token-Klassen den speziellen den Input "wegnehmen".
Verdeutlicht am Beispieltext und den Token-Klassen aus der Angabe:
```python
token_classes = {
    'emoasc': r'[:;=][-^]?[DP()c|\[\]{}]|XD+',
    'punct': r'[,.:;()]',
    'word': r'\w+'
}
```
Wenn nun der Eingabetext `Das ist ein Test. :)` tokenisiert wird, wird in der gegebenen Hierarchie als erstes nach Smileys gesucht, dann nach Satzzeichen, dann nach allen anderen zusammenhängenden Wörtern. In diesem Fall stellt "emoasc" eine speziellere Klasse für "gewisse Kombinationen von Satzzeichen" dar, während "punct" eine allgemeine Klasse für "alle Satzzeichen" darstellt. Würde man die Klassen "emoasc" und "punct" vertauschen, könnten ":" und ")" als einzelne Satzzeichen im Verarbeitungsschritt von "punct" interpretiert werden und nie das Erkennungsmuster für "emoasc" erreichen!

Welche besonderen und interessanten Tokenklassen gibt es in den Eingabetexten? Gibt es Arten von Tokens, die spezifisch für die Art des Eingabetextes sind? 

In [6]:
class Token():
    """Ein Objekt, das ein Token repräsentiert.
    Jedes Token hat einen `text` und eine `token_class`, welche die Klasse (also Art) des Tokens angibt. Außerdem werden
    seine Start- und Endposition im ursprünglichen Text gespeichert"""
    
    def __init__(self, text, token_class, start, end):
        self.text = text
        self.token_class = token_class
        self.start = start
        self.end = end
        
    def __str__(self):
        return self.text
    
    def __repr__(self):
        return f'Token("{self.text}", "{t.token_class}", {t.start}, {t.end})'
    
def debug(text):
    """Diese Funktion tokenisiert den gegebenen Text mit `tokenize()` und gibt zu jedem Token alle Informationen aus"""
    tokens = tokenize(text)
    for t in tokens:
        print(f'"{t.text}":\tclass="{t.token_class}", start="{t.start}", end="{t.end}"')

In [7]:
def tokenize(text):
    """Diese Funktion tokenisiert den in `text` gegebenen Text und gibt eine Liste an Tokens zurück.
    
    Zum Tokenisieren des Eingabetextes werden die regulären Ausdrücke in `token_classes` der Reihe nach mit `re.finditer()`
    auf den Text angewandt. Für jedes Match wird ein neues Token-Objekt erstellt und in einer Liste abgespeichert.
    Alle Matches der Regex werden mithilfe von `re.sub()` aus dem Eingabetext entfernt, bevor die nächste Regex in der
    Hierarchie aufgerufen wird.
    
    Diese Funktion muss prinzipiell nicht geändert werden.
    """
    tokens = []

    for token_class, regex in token_classes.items():
        for match in re.finditer(regex, text):
            tokens.append(
                Token(match[0], token_class, match.start(), match.end())
            )

        text = re.sub(regex, ' ', text)
    
    # Die Tokens sind noch nicht in der richtigen Reihenfolge und müssen erst nach ihrer Startposition sortiert werden
    return sorted(tokens, key = lambda token: token.start)

In [8]:
### Hier Code vervollständigen ###

"""Die einzelnen regulären Ausdrücke zum Tokenisieren der einzelnen Token-Klassen.
Die Schlüssel des Dictionaries sind die Namen der Token-Klassen,
die Werte sind die zugehörigen regulären Ausdrücke.
Beim Tokenisieren werden die Tokenizer der Reihe nach aufgerufen, beginnend mit dem ersten Element im Dictionary."""
token_classes = {
    'emoasc': r'[:;=][-^]?[DP()c|\[\]{}]|XD+',
    'punct': r'[,.:;()]',
    'word': r'\w+'
}

In [9]:
debug('Das ist ein Test. :)')

"Das":	class="word", start="0", end="3"
"ist":	class="word", start="4", end="7"
"ein":	class="word", start="8", end="11"
"Test":	class="word", start="12", end="16"
".":	class="punct", start="16", end="17"
":)":	class="emoasc", start="18", end="20"


## 3. Ausgabe der Texte

Abschließend sollten alle Texte in tokenisierter Form als Datei ausgegeben werden. Ein gängiges Format dafür ist, die Texte als einfache Textdateien auszugeben, wobei immer ein Token pro Zeile steht. D. h.:
```
Ich
habe
alle
meine
Tokens
auf
separaten
Zeilen
!
=D
```
Dies kann einfach über die in Python eingebauten Dateioperationen geschehen, sprich z. B. über `file.write()`. Dabei sollte am besten für jede Eingabedatei eine Ausgabedatei entstehen.

In [10]:
# Hier sollen die tokenisierten Texte ausgegeben werden.